In [ ]:
Konteks: Proses Pengecatan Otomotif
Misalkan kita ingin mengoptimalkan kualitas pengecatan mobil dengan empat faktor:
A: Suhu Oven (°C) → (160, 180)
B: Waktu Pemanasan (menit) → (10, 20)
C: Jenis Cat → (A, B)
D: Tekanan Spray (bar) → (2, 3)

Karena ada 4 faktor dengan 2 level, full factorial membutuhkan  percobaan.
Kita gunakan 1/2 Fraction (Resolution IV, alias  percobaan) untuk mengurangi jumlah eksperimen.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from pyDOE2 import ff2n

In [ ]:
# 1. Buat desain faktorial penuh (2-level untuk 4 faktor)
full_factorial = ff2n(4)  # 16 kombinasi awal
full_df = pd.DataFrame(full_factorial, columns=["A", "B", "C", "D"])

In [ ]:
# 2. Konversi level (-1,1) ke nilai aktual
level_mapping = {
    "A": [160, 180],
    "B": [10, 20],
    "C": ["A", "B"],
    "D": [2, 3]
}

for col in full_df.columns:
    full_df[col] = full_df[col].map({-1: level_mapping[col][0], 1: level_mapping[col][1]})

In [ ]:
# 3. Pilih Fractional Factorial 1/2 (Gunakan Aliasing D = A*B*C)
fractional_df = full_df.iloc[[0, 1, 2, 3, 8, 9, 10, 11]].reset_index(drop=True)

In [ ]:
# 4. Simulasikan respons (misalnya, ketebalan cat dalam mikron)
np.random.seed(42)
fractional_df["Ketebalan_Cat"] = (
    50 + 0.2 * (fractional_df["A"] - 160) - 1.5 * (fractional_df["B"] - 10)
    + np.random.normal(0, 2, len(fractional_df))
)

In [ ]:
# 5. Analisis ANOVA
model = smf.ols('Ketebalan_Cat ~ A + B + C + D', data=fractional_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

In [ ]:
# 6. Visualisasi
plt.figure(figsize=(8,6))
sns.barplot(data=fractional_df, x="A", y="Ketebalan_Cat", hue="B")
plt.title("Pengaruh Suhu Oven dan Waktu Pemanasan terhadap Ketebalan Cat")
plt.xlabel("Suhu Oven (°C)")
plt.ylabel("Ketebalan Cat (µm)")
plt.grid(True)
plt.show()

In [ ]:
# 7. Output hasil
print(fractional_df)
print("\nANOVA Table:\n", anova_table)
print("\nModel Summary:\n", model.summary())